## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-01-18-55-42 +0000,nyt,Fed Dissenters Defend Call for July Rate Cut a...,https://www.nytimes.com/2025/08/01/business/fe...
1,2025-08-01-18-55-12 +0000,nyt,"Live Updates: After a Weak Jobs Report, Trump ...",https://www.nytimes.com/live/2025/07/31/busine...
2,2025-08-01-18-54-41 +0000,nyt,"Trump, Claiming Weak Jobs Numbers Were ‘Manipu...",https://www.nytimes.com/live/2025/07/31/busine...
3,2025-08-01-18-50-00 +0000,wsj,Trump Positions Nuclear Submarines Following C...,https://www.wsj.com/world/russia/trump-positio...
4,2025-08-01-18-48-58 +0000,nyt,"Ross Rowland, 85, Dies; a Railroad Preservatio...",https://www.nytimes.com/2025/08/01/travel/ross...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,79
227,tariffs,30
314,new,21
69,tariff,13
368,up,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
253,2025-07-31-23-00-10 +0000,nypost,Trump struck huge trade deals — but tariffs ar...,https://nypost.com/2025/07/31/us-news/dozens-o...,187
94,2025-08-01-15-15-26 +0000,wapo,Trump’s new tariffs will hit these major tradi...,https://www.washingtonpost.com/business/2025/0...,150
231,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...,149
193,2025-08-01-05-13-22 +0000,bbc,New Trump Tariffs Hit Dozens Of Countries,https://www.bbc.co.uk/iplayer/episode/l0057c0f...,146
215,2025-08-01-02-34-00 +0000,wsj,Lawmakers have introduced bills targeting Chin...,https://www.wsj.com/politics/policy/congress-l...,144


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
253,187,2025-07-31-23-00-10 +0000,nypost,Trump struck huge trade deals — but tariffs ar...,https://nypost.com/2025/07/31/us-news/dozens-o...
231,70,2025-08-01-00-49-00 +0000,wsj,President Trump says he has averted a half-doz...,https://www.wsj.com/politics/policy/trumps-vow...
78,63,2025-08-01-16-06-56 +0000,nypost,Ghislaine Maxwell is quietly moved to cushy ne...,https://nypost.com/2025/08/01/us-news/ghislain...
8,53,2025-08-01-18-45-00 +0000,wsj,Stock Market Today: Dow Falls on Tariff Blitz;...,https://www.wsj.com/livecoverage/jobs-report-t...
37,48,2025-08-01-17-58-15 +0000,nypost,"Anti-Israel NYC Councilwoman Tiffany Caban, do...",https://nypost.com/2025/08/01/us-news/proteste...
117,46,2025-08-01-14-09-33 +0000,nypost,Devil’s Den suspect charged with killing coupl...,https://nypost.com/2025/08/01/us-news/devils-d...
241,41,2025-07-31-23-55-00 +0000,wsj,President Trump is building a Mar-a-Lago style...,https://www.wsj.com/politics/policy/trump-ball...
188,40,2025-08-01-05-44-24 +0000,nypost,Kamala Harris complains ‘the system’ is ‘broke...,https://nypost.com/2025/08/01/us-news/kamala-h...
58,36,2025-08-01-16-49-02 +0000,nypost,"Trump reveals ‘almost 20,000 Russian soldiers ...",https://nypost.com/2025/08/01/us-news/trump-sa...
27,32,2025-08-01-18-20-54 +0000,nypost,Corporation for Public Broadcasting — which fu...,https://nypost.com/2025/08/01/media/corporatio...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
